# get_data

> async wrapper for asyncio requests


In [ ]:
# | default_exp client.get_data

In [ ]:
# | exporti
from typing import Optional, Union

from pprint import pprint

import aiohttp

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.ResponseGetData as rgd


In [ ]:
# | export
async def get_data(
    url: str,
    method: str,
    auth: dmda.DomoAuth,
    content_type: Optional[dict] = None,
    headers: Optional[dict] = None,
    # if no session passed by default will create and close session during execution
    session: Optional[aiohttp.ClientSession] = None,
    body: Union[dict, str, None] = None,
    params: Optional[dict] = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    """async wrapper for asyncio requests"""

    if auth and not auth.token:
        await auth.get_auth_token()

    if headers is None:
        headers = {}

    is_close_session = False
    if session is None:
        is_close_session = True
        session = session or aiohttp.ClientSession()

    headers = {
        "Content-Type": content_type or "application/json",
        "Connection": "keep-alive",
        "accept": "application/json, text/plain",
        **headers,
    }

    if auth:
        headers.update(**auth.auth_header)

    if debug_api:
        pprint(
            {
                "method": method,
                "url": url,
                "headers": headers,
                "json": body,
                "params": params,
            }
        )

    try:
        if headers.get("Content-Type") == "application/json":
            if debug_api:
                print("passing json")

            res = await session.request(
                method=method.upper(),
                url=url,
                headers=headers,
                json=body,
                params=params,
            )

        elif body is not None:
            res = await session.request(
                method=method.upper(),
                url=url,
                headers=headers,
                data=body,
                params=params,
            )

        else:
            res = await session.request(
                method=method.upper(), url=url, headers=headers, params=params
            )

    except Exception as e:
        print(e)

    finally:
        if is_close_session:
            await session.close()

    return await rgd.ResponseGetData._from_aiohttp_response(res, auth = auth)

#### sample implementation of get_data

During execution `get_data()` will attempt to retrieve exchange credentials for an auth token using the `dmda.DomoFullAuth.get_auth_token()` method.

Then the appropriate headers will be passed to the request.


In [ ]:
import os

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-dojo",
    domo_username="jae@onyxreporting.com",
    domo_password=os.environ["DOJO_PASSWORD"]
)

url = "https://domo-dojo.domo.com/api/content/v2/users/me"

try:
    res = await get_data(url=url, method="get", auth=full_auth)
    print(res)

except Exception as e:
    print(e)

ResponseGetData(status=200, response={'id': 1893952720, 'invitorUserId': 587894148, 'displayName': 'Jae Wilson', 'userName': 'jae@onyxreporting.com', 'emailAddress': 'jae@onyxreporting.com', 'avatarKey': 'c605f478-0cd2-4451-9fd4-d82090b71e66', 'accepted': True, 'userType': 'USER', 'modified': 1651692505000, 'created': 1588960518, 'role': 'Admin', 'rights': 63, 'active': True, 'pending': False, 'systemUser': False, 'anonymous': False}, is_success=True)


In [ ]:
#| export
class LooperError(Exception):
    def __init__(self, loop_stage: str, message ):

        super().__init__(f"{loop_stage} - {message}")

In [ ]:
# | export
async def looper(
    auth: dmda.DomoAuth,
    session: aiohttp.ClientSession,
    url,
    offset_params,
    arr_fn: callable,
    loop_until_end: bool = False,
    method="POST",
    body: dict = None,
    fixed_params: dict = None,
    offset_params_in_body: bool = False,
    body_fn=None,
    limit=1000,
    skip = 0,
    maximum=2000,
    debug_api: bool = False,
    debug_loop: bool = False
) -> rgd.ResponseGetData:

    is_close_session = False
    
    if not session:
        session = aiohttp.ClientSession()
        is_close_session = True


    allRows = []
    isLoop = True

    res = None

    if maximum < limit:
        limit = maximum

    while isLoop:
        params = fixed_params or {}

        if offset_params_in_body:
            body[offset_params.get("offset")] = skip
            body[offset_params.get("limit")] = limit

        else:
            params[offset_params.get("offset")] = skip
            params[offset_params.get("limit")] = limit

        if body_fn:
            try:
                body = body_fn(skip, limit)
            
            except Exception as e:
                await session.close()
                raise LooperError(loop_stage = "processing body_fn", message = str(e))
            

        if debug_loop:
            print(
                f"\n🚀 Retrieving records {skip} through {skip + limit} via {url}")
            # pprint(params)

        
        res = await get_data(
            auth=auth,
            url=url,
            method=method,
            params=params,
            session=session,
            body=body,
            debug_api=debug_api,
        )

        if not res.is_success:
            if is_close_session:
                await session.close()
            return res


        try:
            newRecords = arr_fn(res)
        
        except Exception as e:
            await session.close()
            raise LooperError(loop_stage = "processing arr_fn", message = str(e))
        
        allRows += newRecords

        if loop_until_end and len(newRecords) != 0:
            maximum = maximum + limit

        if debug_loop:
            print({"all_rows": len(allRows), "new_records": len(newRecords)})

        if len(allRows) >= maximum or len(newRecords) == 0:
            if debug_loop:
                print(
                    f"\n🎉 Success - {len(allRows)} records retrieved from {url} in query looper\n")

            break

        skip += len(newRecords)
        
        if skip + limit > maximum:
            limit = maximum - len(allRows)

            if debug_loop:
                print(f"skip: {skip}, limit: {limit}")
    
    if is_close_session:
        await session.close()

    return await rgd.ResponseGetData._from_looper(res = res, array = allRows)

In [ ]:
#| hide
import os

session = aiohttp.ClientSession() 

sql = "SELECT * FROM TABLE"
dataset_id = 123

token_auth = dmda.DomoTokenAuth( domo_instance = 'domo-dojo', domo_access_token = os.environ['DOMO_DOJO_ACCESS_TOKEN'] )

url = f"https://{token_auth.domo_instance}.domo.com/api/query/v1/execute/{dataset_id}"

offset_params = {
    "offset": "offset",
    "limit": "limit",
    }

maximum = 10
skip = 0

def body_fn(skip, limit):
    return {"sql": f"{sql} limit {limit} offset {skip}"}

def arr_fn(res : rgd.ResponseGetData):
    rows_ls = res.response.get("rows")
    columns_ls = res.response.get("columns")
    output = []
    for row in rows_ls:
        new_row = {}
        for index, column in enumerate(columns_ls):
            new_row[column] = row[index]
        output.append(new_row)
    return output

res = await looper(
    auth=token_auth,
    method="POST",
    url=url,
    offset_params=offset_params,

    skip=skip,
    maximum=maximum,
    
    arr_fn=arr_fn,
    body_fn=body_fn,
    
    debug_api=True,
    debug_loop=True,
    loop_until_end= False,

    session = session
)

await session.close()

res


🚀 Retrieving records 0 through 10 via https://domo-dojo.domo.com/api/query/v1/execute/123
{'headers': {'Connection': 'keep-alive',
             'Content-Type': 'application/json',
             'accept': 'application/json, text/plain',
             'x-domo-developer-token': 'a9c9c837775a981121fc57b4c78550d28c8784b6b0f4c69c'},
 'json': {'sql': 'SELECT * FROM TABLE limit 10 offset 0'},
 'method': 'POST',
 'params': {'limit': 10, 'offset': 0},
 'url': 'https://domo-dojo.domo.com/api/query/v1/execute/123'}
passing json


ResponseGetData(status=404, response='Not Found', is_success=False)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
